In [10]:
import click
import pandas as pd
from time import time
import os
from google.cloud import storage
import pyspark
from pyspark.sql import SparkSession
from io import BytesIO
import pyspark.sql.functions as F
from pyspark.sql import types
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'dotenv'

In [2]:
#@click.command() # click commands instead of argparse.ArgumentParser()... or sys.argv[n]
#@click.option('--sa_path', help='Path to the service account json file')
#@click.option('--project_id', help='Project ID of you GCP project')
#@click.option('--year', default=2021, help='Year to download')
#@click.option('--bucket', help='Name of the bucket to upload the data')
#@click.option('--color', help='Str of the taxi-color for which data should be extracted')
#@click.option('--month', help='Int of the month to summarize the data for')

sa_path = '../mle-neue-fische-gunnaroeh-0fc41b31bc57.json'
project_id = 'mle-neue-fische-gunnaroeh'
bucket = "01_data_pipeline_project" 
project_id = "mle-neue-fische-gunnaroeh" 
color = "green"
year = 2021
month = 1

In [3]:
### 3. E: Extract the data as a function
def extract_data(sa_path, bucket, color, year, month):
    # Spark Session
    spark = SparkSession.builder \
    .master("local") \
    .appName(f"Pipe-{color}_taxi_{year}-{month:02d}") \
    .getOrCreate()

    # string of the file to be loaded
    file_name = f"ny_taxi/{color}_tripdata_{year}-{month:02d}.parquet"
    
    # Establish connection to GCS-Bucket
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = sa_path
    # Create an instance of the GCS client to communicate with the Cloud
    client = storage.Client()
    
    # Retrieve address/path to the specified bucket and a blob representing the table
    bucket = client.get_bucket(bucket)
    blob = bucket.get_blob(file_name)

    # Download parquet and write it to memory as binary to be accessible
    pq_taxi = blob.download_as_bytes()    
    pq_taxi = BytesIO(pq_taxi)
    
    # read the object in memory as df -> spark-df
    df_taxi = pd.read_parquet(pq_taxi)
    df_taxi.drop("ehail_fee", inplace=True, axis=1)
    df_taxi = spark.createDataFrame(df_taxi)

    # convert the double to float
    for col in df_taxi.columns:
        if df_taxi.schema[col].dataType == types.DoubleType():
            df_taxi = df_taxi.withColumn(col, F.col(col).cast('float'))
    
    return df_taxi, spark

In [4]:
df_taxi,spark = extract_data(sa_path, bucket, color, year, month)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/09 16:27:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df_taxi.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: float (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: float (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- payment_type: float (nullable = true)
 |-- trip_type: float (nullable = true)
 |-- congestion_surcharge: float (nullable = true)



In [6]:
def transform_data(df_taxi, spark):
    # some sql commands
    df_taxi = df_taxi \
        .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
        .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')\
        .withColumnRenamed('PULocationID', 'pickup_location_id')\
        .withColumnRenamed('DOLocationID', 'dropoff_location_id')
    # Temporary SQL Table to be queried
    df_taxi.registerTempTable('df_taxi_temp')
    # Query the revenue
    df_result = spark.sql("""
                    SELECT pickup_location_id AS revenue_zone,
                    date_trunc('day', pickup_datetime) AS revenue_day,
                    date_trunc('month', pickup_datetime) AS month,  
                    SUM(fare_amount) AS revenue_daily_fare,
                    SUM(extra) AS revenue_daily_extra,
                    SUM(mta_tax) AS revenue_daily_mta_tax,
                    SUM(tip_amount) AS revenue_daily_tip_amount,
                    SUM(tolls_amount) AS revenue_daily_tolls_amount,
                    SUM(improvement_surcharge) AS revenue_daily_improvement_surcharge,
                    SUM(total_amount) AS revenue_daily_total_amount,
                    SUM(congestion_surcharge) AS revenue_daily_congestion_surcharge,
                    AVG(passenger_count) AS avg_daily_passenger_count,
                    AVG(trip_distance) AS avg_daily_trip_distance
                    FROM df_taxi_temp
                    GROUP BY revenue_zone, revenue_day, month;
                      """)
    return df_result
        

In [7]:
df_transformed = transform_data(df_taxi, spark)
df_transformed.show(30)

/Users/gunnaroeh/neuefische/mle-data-pipeline-project/.venv/lib/python3.11/site-packages/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)
23/07/09 16:27:50 WARN TaskSetManager: Stage 0 contains a task of very large size (10889 KiB). The maximum recommended task size is 1000 KiB.


+------------+-------------------+-------------------+------------------+-------------------+---------------------+------------------------+--------------------------+-----------------------------------+--------------------------+----------------------------------+-------------------------+-----------------------+
|revenue_zone|        revenue_day|              month|revenue_daily_fare|revenue_daily_extra|revenue_daily_mta_tax|revenue_daily_tip_amount|revenue_daily_tolls_amount|revenue_daily_improvement_surcharge|revenue_daily_total_amount|revenue_daily_congestion_surcharge|avg_daily_passenger_count|avg_daily_trip_distance|
+------------+-------------------+-------------------+------------------+-------------------+---------------------+------------------------+--------------------------+-----------------------------------+--------------------------+----------------------------------+-------------------------+-----------------------+
|          68|2021-01-09 00:00:00|2021-01-01 00:00:0

In [9]:
### 5. L: Load Data onto local Machine PostgreSQL
def load_data(df_transformed, year, month, color):
    load_dotenv()
    # get the Database Credentials
    user = os.getenv('USER')
    pw = os.getenv('PASSWORD')
    host = os.getenv('HOST')
    port = os.getenv('PORT')
    db = os.getenv('DB')
    schema = os.getenv('SCHEMA')
    # some commands to write it into storage in the db
    table_name = f"{color}_revenue_{year}_{month}"
    # Write DataFrame to PostgreSQL
    df_transformed.write.format("jdbc") \
        .option("url", f"jdbc:postgresql://{host}:{port}/{db}") \
        .option("schema", schema) \
        .option("dbtable", table_name) \
        .option("user", user) \
        .option("password", pw) \
        .mode("overwrite") \
        .save()

In [ ]:
load_data(df_transformed, )